In [ ]:
Mon code fonctionne pour n'importe quelle région, il suffit de les noter dans listeRegions. Concernant les numéros de téléphone j'ai utilisé Selenium pour que mon programme clique automatiquement sur le bouton "Voir numéro" de la page de la voiture. Cependant au bout de 3 numéros récupérés, le site LeBonCoin bloque l'accès à ces inf'

In [34]:
#import the library used to query a website
import urllib
import urllib.request
import pandas as pd
#import the Beautiful soup functions to parse the data returned from the website
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException


vendeur = []
prix = []
modele = []
distance = []
annee = []
avgPrice = []
tel = []





# Get Version
def getVersion(text):
    text = text.lower()
    marque = ''
    version = re.findall(('intens|life|zen'), text,flags = 0)
    if len(version)>0:
        marque = version[0]
    else:
        marque = None
    return marque
    
# Get Prix
def getPrice(text):
    text = text.lower()
    prix = 0
    price = re.findall('prix : \"([0-9]+)\"',text,flags = 0)
    if len(price)>0:
        prix = (int)(price[0])
    return prix
    
# Get Distance
def getDistance(text):
    text = text.lower()
    distance = 0
    dist = re.findall('km : \"([0-9]+)\"',text,flags = 0)
    if len(dist)>0:
        distance = (int)(dist[0])
    return distance
    
# Get Annee
def getAnnee(text):
    annee = 0
    year = re.findall('annee : \"([0-9]+)\"',text,flags = 0)
    if len(year)>0:
        annee = (int)(year[0])
    return annee
                     


def getSoup(url):
    page = urllib.request.urlopen(url)
    #Parse the html in the 'page' variable, and store it in Beautiful Soup format
    soup = BeautifulSoup(page)
    return soup

# Get Model Average Price (on prend l'url ici parce que ça dépend de la version et de l'année)
def getModelPrice(version, annee):
    version = str(version)
    annee = str(annee)
    url = 'https://www.lacentrale.fr/cote-auto-renault-zoe-'+version+'-'+annee+'.html'
    page = urllib.request.urlopen(url)
    #Parse the html in the 'page' variable, and store it in Beautiful Soup format
    soup = BeautifulSoup(page)
    text = soup.text.lower()
    price = 0
    r = re.findall("cote brute\n : (\d[^€]*)",text,flags=0)
    if len(r)>0:
        a = r[0].replace(" ","")
        price = (int)(a)
    return price

#### Get links for a region

def getLinks(region):
    #Query the website and return the html to the variable 'page'
    particuliers = "https://www.leboncoin.fr/voitures/offres/"+region+"/?q=Renault%20Zo%E9&f=p"
    professionels = "https://www.leboncoin.fr/voitures/offres/"+region+"/?q=Renault%20Zo%E9&f=c"
    return (particuliers,professionels)

#### Get all pages for one region
def getAllPagesForARegion(url,region,personne):
    liste = []
    #Query the website and return the html to the variable 'page'
    page = urllib.request.urlopen(url)
    #Parse the html in the 'page' variable, and store it in Beautiful Soup format
    soup = BeautifulSoup(page)
    last = soup.find(id = 'last')
    numberPages = 0
    if last != None:
        nbpages = last.get("href")
        numberPages = (int)(nbpages[nbpages.rfind("?o=")+3])
    else:
        numberPages = 1
    for i in range(1,numberPages+1):
        pg = str(i)
        liste.append("https://www.leboncoin.fr/voitures/offres/"+region+"/?o="+pg+"&q=Renault%20Zo%E9&f="+personne)
    return liste

#getAllPagesForARegion(url,'ile_de_france','p')

#### In one page get all cars 
def getAllCarsInAPage(url):
    liste = []
    page = urllib.request.urlopen(url)
    #Parse the html in the 'page' variable, and store it in Beautiful Soup format
    soup = BeautifulSoup(page)
    block = soup.findAll('a', class_ ="list_item clearfix trackable")
    for par in block:
        links = par.get("href")
        liste.append("https:"+links)
    return liste
#getAllCarsInAPage('https://www.leboncoin.fr/voitures/offres/ile_de_france/?o=1&q=Renault%20Zo%E9&f=p')


#### In a car page get the data

def getDataOfACar(url2):
    page2 = urllib.request.urlopen(url2)
    #Parse the html in the 'page' variable, and store it in Beautiful Soup format
    soup2 = BeautifulSoup(page2)
    text = soup2.text.lower()
    prix.append(getPrice(text))
    version = getVersion(text)
    modele.append(version)
    distance.append(getDistance(text))
    year = getAnnee(text)
    annee.append(year)
    avgPrice.append(getModelPrice(version,year))
    tel.append(getPhoneNumber(url2))
    
    
def getAllData(region,personne,url):
    soup = getSoup(url)
    pages = getAllPagesForARegion(url, region, personne)
    for pg in pages:
        cars = getAllCarsInAPage(pg)
        for car in cars:
            getDataOfACar(car)
            vendeur.append(personne)
            
#getDataOfACar("https://www.leboncoin.fr/voitures/1317781865.htm?ca=12_s")

def remplacement(x):
    if x in ['p']:
        x = str.replace(x,x,'particulier')
    elif x in ['c']:
        x = str.replace(x,x,'professionel')
    return x
        

def difference(row):
    if row["DIFFERENCE"]:
        return "Supérieur"
    else:
        return "Inférieur"
    
def getPhoneNumber(url): 
    firefoxPath="/home/maxence/geckodriver"
    profile = webdriver.FirefoxProfile()
    profile.set_preference("webdriver.load.strategy", "unstable")
    profile.update_preferences()
    driver = webdriver.Firefox(firefox_profile=profile,executable_path=firefoxPath)
    driver.set_page_load_timeout(2)
    try:
        driver.get(url)
    except TimeoutException:
        driver.execute_script("window.stop();")
    try:
        submit = driver.find_element_by_class_name("button-orange.large.phoneNumber.trackable")
    except:
        numero_tel = "None"
    try:
        submit.click()
    except:
        print("")
    driver.implicitly_wait(2)
    number = driver.find_elements_by_css_selector('span.phone_number:nth-child(1) > a:nth-child(1)')
    if number != None:
        numero_tel = ""
        for num in number:
            numero_tel = numero_tel + num.text
        if len(numero_tel) < 2:
            numero_tel = "None"
    else:
        numero_tel = "None"
    driver.close()
    return numero_tel

    
    
def insertDataIntoDF():
    df=pd.DataFrame(modele,columns = ['MODELE'])
    df["ANNEE"] = annee
    df["DISTANCE"] = distance
    df["VENDEUR"] = vendeur
    df['VENDEUR'] = df['VENDEUR'].apply(lambda x : remplacement(x) )
    df["PRIX"] = prix
    df["avgPrice"] = avgPrice
    df["DIFFERENCE"] = df["PRIX"]>df["avgPrice"]
    df.loc[df["PRIX"] >= df["avgPrice"], "DIFFERENCE"] = "Supérieur"
    df.loc[df["PRIX"] < df["avgPrice"], "DIFFERENCE"] = "Inférieur"
    df["TELEPHONE"] = tel
    return df
    

def mainMethodRegion(region):
    (par,prof) = getLinks(region)
    getAllData(region,"p", par)
    getAllData(region,"c", prof)

def mainMethod(liste):
    for region in liste:
        mainMethodRegion(region)
    df = insertDataIntoDF()
    df.to_csv('/home/maxence/renaultZoe.csv')
    return df

In [35]:
# = ["ile_de_france", "aquitaine", "provence_alpes_cote_d_azur"]
listeRegions = ["alsace"]
df = mainMethod(listeRegions)
df.head()

/home/maxence/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/maxence/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


,MODELE,ANNEE,DISTANCE,VENDEUR,PRIX,avgPrice,DIFFERENCE,TELEPHONE
0,life,2013,900000,particulier,7500,8790,Inférieur,06 68 14 46 92
1,life,2014,35000,particulier,8500,10593,Inférieur,06 85 84 86 45
2,None,2016,3950,particulier,12500,0,Supérieur,None
3,zen,2013,29500,particulier,8600,8999,Inférieur,06 11 40 90 39
4,intens,2014,18500,particulier,9900,10128,Inférieur,None
